install libraries

In [1]:
!pip install -q transformers
!pip install -q "datasets<=2.18.0"
!pip install -q sentence-transformers
!pip install -q langchain
!pip install -q langchain_community
!pip install -q torch
!pip install -q numpy
!pip install -q faiss-gpu

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.llms import HuggingFacePipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset

# List available datasets and configurations
imdb_dataset = load_dataset("imdb")

# Print available splits to ensure correct loading
print(imdb_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [3]:
import re

# Define the word or phrase you want to filter by
specific_word = "Star Wars"

# Define a function to filter texts containing the specific word or phrase
def filter_texts(example):
    return bool(re.search(rf'\b{specific_word}\b', example['text'], re.IGNORECASE))

# Apply the filter to the train and test datasets
filtered_train_dataset = imdb_dataset['train'].filter(filter_texts)
filtered_test_dataset = imdb_dataset['test'].filter(filter_texts)
filtered_unsupervised_dataset = imdb_dataset['unsupervised'].filter(filter_texts)

# Verify the filtered dataset
print(filtered_train_dataset)
print(filtered_test_dataset)
print(filtered_unsupervised_dataset)

print(filtered_train_dataset[0])
print(filtered_test_dataset[0])
print(filtered_unsupervised_dataset[0])

Filter:   0%|          | 0/25000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 159
})
Dataset({
    features: ['text', 'label'],
    num_rows: 153
})
Dataset({
    features: ['text', 'label'],
    num_rows: 306
})
{'text': 'As a kid I did think the weapon the murderer wielded was cool, however I was a kid and so I was a bit dumb. Even as a dumb kid though the movies plot was stupid and a bit boring when the killer was not using his light knife to kill people. What amazes me is that the movie has a really solid cast in it. What script did they read when agreeing to be in this movie as it is most assuredly boring and only a means to show off a light saber on a very small scale. The plot at times is incomprehensible and the end is totally chaotic. The whole film seems to rotate around aliens and the one weapon. The plot has two kids and some dude having an alien encounter, flash years later and there seems to be a return as it were in the mix. Dead animals and such to be explored and for some reason the one du

In [4]:
from datasets import concatenate_datasets

combined_dataset = concatenate_datasets([filtered_train_dataset, filtered_test_dataset, filtered_unsupervised_dataset])

print(combined_dataset)

print(combined_dataset[0])

Dataset({
    features: ['text', 'label'],
    num_rows: 618
})
{'text': 'As a kid I did think the weapon the murderer wielded was cool, however I was a kid and so I was a bit dumb. Even as a dumb kid though the movies plot was stupid and a bit boring when the killer was not using his light knife to kill people. What amazes me is that the movie has a really solid cast in it. What script did they read when agreeing to be in this movie as it is most assuredly boring and only a means to show off a light saber on a very small scale. The plot at times is incomprehensible and the end is totally chaotic. The whole film seems to rotate around aliens and the one weapon. The plot has two kids and some dude having an alien encounter, flash years later and there seems to be a return as it were in the mix. Dead animals and such to be explored and for some reason the one dude gets the weapon of the aliens and proceeds to use it to go on a very light killing spree. Seriously, you just have to wonder 

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 100,
    length_function = len,
    is_separator_regex=False,
)

# Extract texts from the dataset
texts = combined_dataset["text"]

# Concatenate all texts into a single string
concatenated_text = " ".join(texts)

texts2 = text_splitter.create_documents([concatenated_text])

print(texts2[:2])
# print(texts2[1])


[Document(page_content='As a kid I did think the weapon the murderer wielded was cool, however I was a kid and so I was a bit dumb. Even as a dumb kid though the movies plot was stupid and a bit boring when the killer was not using his light knife to kill people. What amazes me is that the movie has a really solid cast in it. What script did they read when agreeing to be in this movie as it is most assuredly boring and only a means to show off a light saber on a very small scale. The plot at times is incomprehensible and the end is totally chaotic. The whole film seems to rotate around aliens and the one weapon. The plot has two kids and some dude having an alien encounter, flash years later and there seems to be a return as it were in the mix. Dead animals and such to be explored and for some reason the one dude gets the weapon of the aliens and proceeds to use it to go on a very light killing spree. Seriously, you just have to wonder why this movie was made, if you are going to have 

In [6]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)

vector_store = FAISS.from_documents(texts2, embedder)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
query = "What is Star Wars?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

be any actual emphasis on plot, it just being an excuse to glue one over-long action set piece to another, so the intrigue disappears fast and the themes and contexts are given a back seat to (admittedly impressive) CGI. It seems that George Lucas thought that what made a good film was as much soulless eye candy as possible.<br /><br />Star wars was once a trailblazer in Science fiction and its originality gave most of the future Blockbusters the templates for their scenes, and what was originally Star Wars innovation became cliché at the hands of other films. Now it is Star Wars using the same plot lines we have seen over and over again, copying others and Episodes 1, 2 and 3 in a lazy attempt to seem interesting.<br /><br />You do not need spectacle to create a good film you need drama and humanity both of which this film sorely lacks. In my opinion the best Light Sabre Fight of the series was the Showdown between Darth Vader and Obi-Wan Kenobi. The fight didn't have any flipping or


In [8]:
query = "Who is darth vader?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

to working alongside the Rebels to destroy an all new Death Star that nears completion. This time the Emperor himself is overseeing the final stages of construction. The Empire intends to crush the Rebellion once and for all, while the Emperor himself schemes to bring the now powerful Skywalker to his side to work alongside (or is that replace?) Darth Vader. The Emperor is a different kind of evil for this film, less cunning than Governor Tarkin (Peter Cushing) from "Star Wars", more deeply psychologically dark than anything else. Played brilliantly by Ian McDiarmid, the Emperor is just one of those characters you love to hate.<br /><br />All the other actors are well entrenched in their roles. Hamill surprises as the more wisened Luke, making his character's progression from whiny teenager, impatient student to enlightened warrior one of the few real character developments of the series. Ford's role is waring thin, as all his charm and charisma was spent in the first two films -- he
a

In [9]:
query = "Who is luke skywalker's father?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

is that Yoda is too old and sick, since he already has 900 years old, and before he dies, Yoda says to Luke that he does not need more training,but to really be a Jedi, he must fight with Vader again. He confirms to Luke that Vader is Luke's dad, and that there is another Skywalker besides Luke. In his last moments, Yoda asks to Luke to remember his advices about the temptation of the dark side, and to Luke transmit his Jedi knowledge to other people. When Yoda dies,Obi wan's spirit shows up to Luke and tells him that Luke's father killed his good side Anakin to become Darth Vader, and also that he is more machine than a man since he became a sith. Luke stays worried about killing his own dad, and says that he feels that his father still has kindness. Obi Wan tells Luke that his twin sister is Leia, and says the reasons why Luke and Leia were separated since babies. He gives his last advice to Luke saying that if he refuses to kill Vader, the emperor will win the war.<br /><br />At
Fat

In [10]:
query = "Why were the sequals worse than the originals or the prequals?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

innovative for it's time, and that it has been copied very often, but maybe that's exactly why the whole thing sometimes seems a little below it's reputation. There are tough, rebellious youths and pink-haired squeaky cute computer geniuses all over the place on the anime market. The fact that they're all female is hardly a novelty anymore either.<br /><br />That might be the main problem here; time hasn't been too kind to Bubblegum Crisis. Those who first saw the series and became loyal followers in the 80's had every reason to be impressed back then. Alas, by now, the same formula has appeared so often, be it under different guises, that there's little left to be amazed about for one watching this series for the first time now. Somehow BGC doesn't really manage to hold up a feeling that it's the founding father of a whole generation of anime series.<br /><br />Animation quality has aged a bit better though, with only the first episode looking genuinely old hat by now. Episodes seven


In [11]:
query = "Were the prequals worse than the orginals?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

calculator once. I'm never quite sure whether the Cylons had an organic counterpart who created them, but it doesn't really matter.<br /><br />In many ways this show is a shallow imitation of "Star Wars," and this can be seen in the cast of characters. Particularly Starbuck (Dirk Benedict), who tries so hard to be Han Solo, but only comes off as petulant, sexist, and rather pathetic. Apollo is the noble, Luke Skywalker hero type, and as such doesn't have a personality. Starbuck has a faithful black sidekick, Boomer, who seems to be much more competent than Starbuck. Adama also has a faithful black sidekick, called Colonel Tigh, who at one point is reduced to picking up everyone's laundry. Take that, Civil Rights Movement!<br /><br />The destruction of the human colonies is a brief, and amazingly painless affair. A park is strafed by Cylon ships, and a small dog is killed (though the actual death is not shown, so we are left with some hope). Soon enough, our heroes have forgotten all
US

In [12]:
# huggingface_embeddings = HuggingFaceBgeEmbeddings(
#     model_name="BAAI/bge-small-en-v1.5",  # alternatively use "sentence-transformers/all-MiniLM-l6-v2" for a light and faster experience.
#     model_kwargs={'device':'cpu'},
#     encode_kwargs={'normalize_embeddings': True}
# )

In [13]:
# sample_embedding = np.array(huggingface_embeddings.embed_query(texts2[0].page_content))
# print("Sample embedding of a document chunk: ", sample_embedding)
# print("Size of the embedding: ", sample_embedding.shape)

In [14]:
# vectorstore = FAISS.from_documents(texts2, huggingface_embeddings)

In [15]:
# query = """What is Star Wars."""
#          # Sample question, change to other questions you are interested in.
# relevant_documents = vectorstore.similarity_search(query)
# print(f'There are {len(relevant_documents)} documents retrieved which are relevant to the query. Display the first one:\n')
# print(relevant_documents[0].page_content)